In [1]:
# Jenny branch
import numpy as np
import tensorflow as tf 
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, Flatten, BatchNormalization
from sklearn.model_selection import KFold

Using TensorFlow backend.


In [2]:
def load_data(filename, skiprows = 1):
    """
    Function loads data stored in the file filename and returns it as a numpy ndarray.
    
    Inputs:
        filename: given as a string.
        
    Outputs:
        Data contained in the file, returned as a numpy ndarray
    """
    return np.loadtxt(filename, skiprows=skiprows, delimiter=' ')

In [3]:
# Loads training dataset
training = load_data('data/training_data.txt', 1)
X_train = training[:, 1:]
Y_train = training[:,0]

In [4]:
# Loads testing dataset
# There is no label for testing set 
X_test = load_data('data/test_data.txt', 1)

In [5]:
# Converting Y values to one hot vector
num_classes = 2
Y_train = keras.utils.to_categorical(Y_train, num_classes=num_classes)


In [6]:
# Input size
n_train = X_train.shape[0]
n_words = X_train.shape[1]
n_test = Y_train.shape[0]


In [7]:
# For 1D Convolution, change dimension of input 
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)

In [8]:
    
## Create your own model here given the constraints in the problem
model = Sequential()

model.add(Conv1D(5, 3, padding="same", input_shape=(n_words, 1, )))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Conv1D(5, 2, padding="same"))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.1))

## Once you one-hot encode the data labels, the line below should be predicting probabilities of each of the 2 classes
model.add(Dense(2))
model.add(Activation('softmax'))

## Printing a summary of the layers and weights in your model
model.summary()

## In the line below we have specified the loss function as 'mse' (Mean Squared Error) because in the above code we did not one-hot encode the labels.
## In your implementation, since you are one-hot encoding the labels, you should use 'categorical_crossentropy' as your loss.
## You will likely have the best results with RMS prop or Adam as your optimizer.  In the line below we use Adadelta
model.compile(loss='categorical_crossentropy',optimizer='RMSprop', metrics=['accuracy'])





_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1000, 5)           20        
_________________________________________________________________
batch_normalization_1 (Batch (None, 1000, 5)           20        
_________________________________________________________________
activation_1 (Activation)    (None, 1000, 5)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000, 5)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1000, 5)           55        
_________________________________________________________________
activation_2 (Activation)    (None, 1000, 5)           0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 500, 5)            0         
__________

In [ ]:
kf = KFold(n_splits = 5)
inds = [ind for ind in kf.split(X_train, Y_train)]
        
total_train_acc = 0
total_val_acc = 0
    
# perform 5-fold validation
for i in range(0,5):
    traini, vali = inds[i]
    fit = model.fit(X_train[traini], Y_train[traini], batch_size=128, epochs=10, verbose=1)
            
    # Compute accuracy.
    train_acc = model.evaluate(X_train[traini], Y_train[traini], verbose=1)
    val_acc = model.evaluate(X_train[vali], Y_train[vali], verbose=1)
            
    total_train_acc += train_acc[1]
    total_val_acc += val_acc[1]
        
print("average training accuracy", total_train_acc / 5.)
print("average val accuracy", total_val_acc / 5.)


Epoch 1/10
16000/16000 [==============================] - 13s 815us/step - loss: 0.0868 - acc: 0.9706
Epoch 2/10
16000/16000 [==============================] - 14s 855us/step - loss: 0.0731 - acc: 0.9738
Epoch 3/10
16000/16000 [==============================] - 13s 785us/step - loss: 0.0670 - acc: 0.9776
Epoch 4/10
16000/16000 [==============================] - 13s 802us/step - loss: 0.0627 - acc: 0.9792
Epoch 5/10
16000/16000 [==============================] - 12s 744us/step - loss: 0.0557 - acc: 0.9813
Epoch 6/10
16000/16000 [==============================] - 12s 749us/step - loss: 0.0544 - acc: 0.9829
Epoch 7/10
16000/16000 [==============================] - 12s 749us/step - loss: 0.0466 - acc: 0.9842
Epoch 8/10
16000/16000 [==============================] - 13s 806us/step - loss: 0.0480 - acc: 0.9846
Epoch 9/10
16000/16000 [==============================] - 13s 812us/step - loss: 0.0468 - acc: 0.9843
Epoch 10/10
4000/4000 [==============================] - 1s 325us/step
Epoch 1/10


In [ ]:
## Printing the accuracy of our model, according to the loss function specified in model.compile above
score = model.evaluate(X_train, Y_train, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
prediction = model.predict(X_test, verbose=1)
print(prediction)

zeros = prediction[:, 0]
ones  = prediction[:, 1]

prediction = (zeros < ones).astype(int)

print(prediction)
print("prediction shape is: {}".format(prediction.shape))

output = "Id,Prediction\n"
for i in range(prediction.shape[0]):
    output = output + ("{0},{1}\n".format(i + 1, prediction[i]))

file = open('predictions.csv','w') 
file.write(output)
file.close()
